## Typical baselines
Проверим три основных идеи:
* Порекомендуем пользователю его посленюю покупку
    - Score: 0.32772 (mean) - 0.33847 (level=1), 0.33623 (level=2), 0.30845 (level=3), 
* Для каждого пользователя найдем самые часто покупаемые товары и порекомендуем их
    - Score: 0.32311 (mean) - 0.34178 (level=1), 0.33146 (level=2), 0.29609 (level=3)
* Каждому пользователю порекомендуем просто топ популярных товаров
    - Score: 0.37599 (mean) - 0.36759 (level=1), 0.37323 (level=2), 0.38714 (level=3)
    
! Можно использовать режим "Run all"

In [1]:
%%time
%pylab inline
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score
tqdm.pandas()

Populating the interactive namespace from numpy and matplotlib
Wall time: 2.62 s


c:\envs\kate\lib\site-packages\tqdm\std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_csv("data/main.csv")

In [4]:
df.rename(columns={"order_completed_at":"time"}, inplace=True) # rename "order_completed_at" column to "time"
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S") # "time" column to datetime type

In [9]:
# в одно и то же время покупку могли совершить несколько клиентов, поэтому в функции duplicates_to_count
# стоит сделать группировку по user_id и time

sum(df.groupby('time').agg(unique_users = ('user_id', 'nunique')).unique_users > 1)

1245

In [12]:
# например..

df[df.time == '2020-04-22 07:13:56']

,user_id,time,cart
1278147,687,2020-04-22 07:13:56,432
1278148,12124,2020-04-22 07:13:56,422
1278149,687,2020-04-22 07:13:56,84
1278150,687,2020-04-22 07:13:56,427
1278151,6815,2020-04-22 07:13:56,440
...,...,...,...
1278215,687,2020-04-22 07:13:56,398
1278216,6815,2020-04-22 07:13:56,377
1278217,6815,2020-04-22 07:13:56,229
1278218,687,2020-04-22 07:13:56,413


Изменила группировку в функции duplicates_to_count, внесла маленькую корректировку в функцию count_to_duplicates. В функции make_train_targets поменяла last() на max() при выборке времени последней покупки (на случай, если данные не отсортированы), подправила мердж user_last_time и df (соединение должно происходить ко колонкам user_id и time).

In [17]:
def duplicates_to_count(t): #Заменяет повторяющиеся товары в корзине на одну запись с указанием количества
    return t.groupby(['user_id', 'time'])['cart'].value_counts() \
                                                  .to_frame() \
                                                  .rename(columns={"cart":"count"}) \
                                                  .reset_index()

def count_to_duplicates(t): #Обратная операция
    g = t.copy()
    g["to_explode"] = g["count"].apply(lambda x: [i for i in range(x)])
    g = g.explode("to_explode") \
         .drop(columns=["count", "to_explode"])
    return g

# Создает train датасет и ответы для него
# lvevel=1 означает, что для формирования ответов берётся последняя покупка, level=2 - предпоследняя и так далее 
def make_train_targets(t, level=1): 
    user_last_time = t.groupby(["user_id"])["time"].max().to_frame().reset_index()
    user_last_time["last_buy"] = 1
    
    train = pd.merge(t, user_last_time, on=["time", "user_id"], how="left")
    train = train[train["last_buy"] != 1]
    train.drop(columns=["last_buy"], inplace=True)
    
    if level >= 2:
        return make_train_targets(train, level-1)
    
    user_last_time.drop(columns=["last_buy"], inplace=True)
    
    user_last_carts = pd.merge(user_last_time, t, on=["user_id", "time"], how="left")
    
    skeleton = make_skeleton(t)
    targets = pd.merge(skeleton, user_last_carts.drop(columns=["time"]), on=["user_id","cart"], how="left")
    targets.fillna(0, inplace=True)
    targets["count"] = targets["count"].apply(lambda x: x if x <= 1 else 1).astype(int)
    targets.rename(columns={"count":"target"}, inplace=True)
    return train, targets

def make_skeleton(t): # Создает все комбинации user_id и cart, которые пользователь когда-либо покупал
    return t.groupby("user_id")["cart"].unique().to_frame().reset_index().explode("cart")
    

In [18]:
df = duplicates_to_count(df)

In [19]:
df.head()

,user_id,time,cart,count
0,0,2020-07-19 09:59:17,14,1
1,0,2020-07-19 09:59:17,20,1
2,0,2020-07-19 09:59:17,57,1
3,0,2020-07-19 09:59:17,82,1
4,0,2020-07-19 09:59:17,379,1


### User's last cart

В функцию user_last_cart_baseline внесла аналогичные правки, что и в make_train_targets.

In [20]:
def user_last_cart_baseline(t, skeleton):
    user_last_time = t.groupby(["user_id"])["time"].max().to_frame().reset_index()
    user_last_carts = pd.merge(user_last_time, t, on=["user_id", "time"], how="left") 
    res = pd.merge(skeleton, user_last_carts.drop(columns=["time"]), on=["user_id","cart"], how="left")\
        .fillna(0).rename(columns={"count":"predict"})
    res["predict"] = res["predict"].progress_apply(lambda x: x if x <= 1 else 1).astype(int)
    return res

In [21]:
train, targets = make_train_targets(df, level=1)

In [22]:
train.head()

,user_id,time,cart,count
0,0,2020-07-19 09:59:17,14,1
1,0,2020-07-19 09:59:17,20,1
2,0,2020-07-19 09:59:17,57,1
3,0,2020-07-19 09:59:17,82,1
4,0,2020-07-19 09:59:17,379,1


In [23]:
targets.head()

,user_id,cart,target
0,0,14,0
1,0,20,0
2,0,57,1
3,0,82,0
4,0,379,0


In [24]:
train_res = user_last_cart_baseline(train, targets)

100%|███████████████████████████████████████████████████████████████████| 1117600/1117600 [00:00<00:00, 1248222.99it/s]


In [25]:
train_res.head()

,user_id,cart,target,predict
0,0,14,0,1
1,0,20,0,0
2,0,57,1,1
3,0,82,0,1
4,0,379,0,1


In [26]:
f1_score(train_res["target"], train_res["predict"])

0.3393176623703617

### User's top personal recommendations

Упростила функцию.

In [85]:
def user_top_k_personal_rec_baseline(t, skeleton, k=10):
    g = count_to_duplicates(t).groupby("user_id")["cart"].value_counts().to_frame().rename(columns={"cart":"count"})\
        .groupby("user_id")["count"].head(10).to_frame().reset_index().drop(columns=["count"])
    g['predict'] = 1
    res = skeleton.copy()
    res = res.merge(g, how='left', on=['user_id', 'cart'])
    res['predict'] = res['predict'].fillna(0).astype(int)
    
    return res

In [86]:
train, targets = make_train_targets(df, level=1)

In [87]:
train_res = user_top_k_personal_rec_baseline(train, targets)

In [88]:
train_res.head()

,user_id,cart,target,predict
0,0,14,0,1
1,0,20,0,1
2,0,57,1,1
3,0,82,0,1
4,0,379,0,1


In [89]:
f1_score(train_res["target"], train_res["predict"])

0.34319546434025816

### Top popular items

Упростила функцию.

In [65]:
def top_k_popular_baseline(t, skeleton, k=15):
    top_k_items = t.groupby("cart")["count"].sum().to_frame().reset_index() \
                   .sort_values("count", ascending=False).head(k)["cart"].tolist()
    
    res = skeleton.copy()
    res['predict'] = np.where(res.cart.isin(top_k_items), 1, 0)
    
    return res

In [66]:
train, targets = make_train_targets(df, level=1)

In [67]:
train_res = top_k_popular_baseline(train, targets)

In [68]:
f1_score(train_res["target"], train_res["predict"])

0.36758787992167213